## Necessary imports 

In [2]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from utils import load_old_model, get_image_features, get_result
import pandas as pd
from PIL import Image
import numpy as np
from annoy import AnnoyIndex
import random
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

## Config GPU

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

## Load necessary files from disk

In [ ]:
with open('./full_catalog_images.pkl', 'rb') as f:
    full_catalog_images = pickle.load(f)

In [ ]:
with open('./street_images.pkl', 'rb') as f:
    street_images = pickle.load(f)

In [ ]:
with open('./bbox_mappings_streetImg.pkl', 'rb') as f:
    bbox_mappings = pickle.load(f)

## Load trained model

In [ ]:
triplet_model = load_old_model('Your model path')

### Generate embedding space for the test images

In [ ]:
embeddings = {}
for index, row in full_catalog_images.iterrows():
    if index % 500 == 0:
        print(str(index) + ' of ' + str(len(full_catalog_images)))
    img_path = row['file_path']
    img = np.array(Image.open(img_path))
    image_features = get_image_features(triplet_model, img, N=2048)
    embeddings[index] = image_features

## Annoy Index

In [ ]:
search_index = AnnoyIndex(N, metric='euclidean')
for i, embedding in embeddings.items():
    search_index.add_item(i, embedding)  
search_index.build(50)

## Visualization

In [ ]:
path = 'path to your street images'

for i, image in enumerate(street_images):
    
    q_img = os.path.join(Path, image)
    if os.path.isfile(q_img):
        bbox = bbox_mappings[i]
        
        left = bbox['left']
        top = bbox['top']
        right = left + bbox['width']
        bottom = top + bbox['height']
        
        query_img = Image.open(q_img)
        
        query_crop = query_img.crop((left, top, right, bottom))
        query = np.array(query_crop.resize((300,300), Image.NEAREST))
        
        image_embeddings = get_image_features(triplet_model, query_array)
        kNN_results = get_result(image_embeddings, search_index, k=10, query_index=q_index)
        print(kNN_results)
        
        fig, ax = plt.subplots(1, 11, figsize=(25, 25))
        ax[0].imshow(img)
        ax[0].set_title(f'query_image, id={i}')

        for i, ax in enumerate(ax.flatten()[1:]):
            ax.imshow(street_images[i])
